# Regression Exercise 

California Housing Data

This data set contains information about all the block groups in California from the 1990 Census. In this sample a block group on average includes 1425.5 individuals living in a geographically compact area. 

The task is to aproximate the median house value of each block from the values of the rest of the variables. 

 It has been obtained from the LIACC repository. The original page where the data set can be found is: http://www.liaad.up.pt/~ltorgo/Regression/DataSets.html.
 

The Features:
 
* housingMedianAge: continuous. 
* totalRooms: continuous. 
* totalBedrooms: continuous. 
* population: continuous. 
* households: continuous. 
* medianIncome: continuous. 
* medianHouseValue: continuous. 

## The Data

** Import the cal_housing_clean.csv file with pandas. Separate it into a training (70%) and testing set(30%).**

In [307]:
import pandas as pd

In [308]:
house_data = pd.read_csv('cal_housing_clean.csv')

In [309]:
house_data.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [310]:
house_data.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
housingMedianAge,20640.0,28.639486,12.585558,1.0000,18.0000,29.0000,37.00000,52.0000
totalRooms,20640.0,2635.763081,2181.615252,2.0000,1447.7500,2127.0000,3148.00000,39320.0000
totalBedrooms,20640.0,537.898014,421.247906,1.0000,295.0000,435.0000,647.00000,6445.0000
population,20640.0,1425.476744,1132.462122,3.0000,787.0000,1166.0000,1725.00000,35682.0000
households,20640.0,499.539680,382.329753,1.0000,280.0000,409.0000,605.00000,6082.0000
medianIncome,20640.0,3.870671,1.899822,0.4999,2.5634,3.5348,4.74325,15.0001
medianHouseValue,20640.0,206855.816909,115395.615874,14999.0000,119600.0000,179700.0000,264725.00000,500001.0000


In [311]:
X_data = house_data.drop('medianHouseValue',axis=1)

In [312]:
y_data = house_data['medianHouseValue']

In [313]:
from sklearn.model_selection import train_test_split

In [314]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.3, random_state=101)

### Scale the Feature Data

** Use sklearn preprocessing to create a MinMaxScaler for the feature data. Fit this scaler only to the training data. Then use it to transform X_test and X_train. Then use the scaled X_test and X_train along with pd.Dataframe to re-create two dataframes of scaled data.**

In [315]:
from sklearn.preprocessing import MinMaxScaler

In [316]:
scaler = MinMaxScaler()

In [317]:
scaler.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [318]:
X_train = pd.DataFrame(data=scaler.transform(X_train),columns = X_train.columns,index=X_train.index)

In [319]:
X_test = pd.DataFrame(data=scaler.transform(X_test),columns=X_test.columns,index=X_test.index)

### Create Feature Columns

** Create the necessary tf.feature_column objects for the estimator. They should all be trated as continuous numeric_columns. **

In [320]:
house_data.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome', 'medianHouseValue'],
      dtype='object')

In [321]:
import tensorflow as tf

In [322]:
feature_columns = []
for col in X_train.columns:
    tf_feature_column = tf.feature_column.numeric_column(col)
    feature_columns.append(tf_feature_column)

** Create the input function for the estimator object. (play around with batch_size and num_epochs)**

In [323]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=10,num_epochs=1000,shuffle=True)

** Create the estimator model. Use a DNNRegressor. Play around with the hidden units! **

In [324]:
model = tf.estimator.DNNRegressor(hidden_units=[6,6,6],feature_columns=feature_columns)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/sr/9k0jqkfs33q22ppyt_t7yjt9716826/T/tmp3he54w2k', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a11cd3940>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


##### ** Train the model for ~1,000 steps. (Later come back to this and train it for more and check for improvement) **

In [325]:
model.train(input_fn=input_func,steps=1000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/sr/9k0jqkfs33q22ppyt_t7yjt9716826/T/tmp3he54w2k/model.ckpt.
INFO:tensorflow:loss = 3.6984e+11, step = 1
INFO:tensorflow:global_step/sec: 672.603
INFO:tensorflow:loss = 7.38995e+11, step = 101 (0.150 sec)
INFO:tensorflow:global_step/sec: 844.347
INFO:tensorflow:loss = 3.69922e+11, step = 201 (0.118 sec)
INFO:tensorflow:global_step/sec: 850.447
INFO:tensorflow:loss = 5.00073e+11, step = 301 (0.118 sec)
INFO:tensorflow:global_step/sec: 803.497
INFO:tensorflow:loss = 5.05466e+11, step = 401 (0.125 sec)
INFO:tensorflow:global_step/sec: 880.027
INFO:tensorflow:loss = 4.69884e+11, step = 501 (0.113 sec)
INFO:tensorflow:global_step/sec: 728.713
INFO:tensorflow:loss = 3.26967e+11, step = 601 (0.137 sec)
INFO:tensorflow:global_step/sec: 733.983
INFO:tensorflow:loss = 6.48326e+11, step = 701 (0.138 sec)
INFO:tensorflow:global_step/sec: 789.703
INFO:tensorflow:loss = 4.43496e+11, step = 801 (0.12

** Create a prediction input function and then use the .predict method off your estimator model to create a list or predictions on your test data. **

In [326]:
predict_input_func = tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=10,num_epochs=1,shuffle=False)

In [327]:
pred_gen = model.predict(predict_input_func)

In [328]:
predictions_list = list(pred_gen)

INFO:tensorflow:Restoring parameters from /var/folders/sr/9k0jqkfs33q22ppyt_t7yjt9716826/T/tmp3he54w2k/model.ckpt-1000


** Calculate the RMSE. You should be able to get around 100,000 RMSE (remember that this is in the same units as the label.) Do this manually or use [sklearn.metrics](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) **

In [329]:
final_preds = []
for pred in predictions_list:
    final_preds.append(pred['predictions'])

In [330]:
from sklearn.metrics import mean_squared_error

In [331]:
mean_squared_error(y_test,final_preds)**0.5

186452.29453704608

# Great Job!